In [1]:
!pip install nba_api

     ---------------------------------------- 0.0/261.7 kB ? eta -:--:--
     - -------------------------------------- 10.2/261.7 kB ? eta -:--:--
     ---- -------------------------------- 30.7/261.7 kB 435.7 kB/s eta 0:00:01
     ---- -------------------------------- 30.7/261.7 kB 435.7 kB/s eta 0:00:01
     ---- -------------------------------- 30.7/261.7 kB 435.7 kB/s eta 0:00:01
     ----- ------------------------------- 41.0/261.7 kB 140.3 kB/s eta 0:00:02
     ---------- -------------------------- 71.7/261.7 kB 231.0 kB/s eta 0:00:01
     --------------- -------------------- 112.6/261.7 kB 312.2 kB/s eta 0:00:01
     --------------- -------------------- 112.6/261.7 kB 312.2 kB/s eta 0:00:01
     ------------------- ---------------- 143.4/261.7 kB 303.9 kB/s eta 0:00:01
     -------------------------- --------- 194.6/261.7 kB 380.0 kB/s eta 0:00:01
     ------------------------------ ----- 225.3/261.7 kB 404.2 kB/s eta 0:00:01
     -------------------------------- --- 235.5/261.7

In [33]:
import pandas as pd
import numpy as np
#import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

### Load Data
Fetch data from a free NBA Stat API

In [2]:
### Get All teams
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

In [3]:
from nba_api.stats.endpoints import leaguegamefinder

### Get games of current season
games_per_team = []
for team in nba_teams:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team['id'])
    games = gamefinder.get_data_frames()[0][['SEASON_ID','TEAM_ABBREVIATION','GAME_ID','GAME_DATE']]
    games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
    games_per_team.append(games[(games.SEASON_ID == "22023") & (games.GAME_DATE >= "2023-10-24")])

games_per_team[0].head()


,SEASON_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE
0,22023,ATL,0022300393,2023-12-23
1,22023,ATL,0022300384,2023-12-22
2,22023,ATL,0022300371,2023-12-20
3,22023,ATL,0022300352,2023-12-18
4,22023,ATL,0022300336,2023-12-16


In [4]:
from nba_api.stats.endpoints import boxscoreadvancedv3

### Get the mean of Offense, Defense and Net rating
target_stats = []
for team in games_per_team:
    target_stats.append({'team_abbreviation':team.TEAM_ABBREVIATION.unique()[0],
                         'offensive_rating':[], 
                         'defensive_rating':[],
                         'net_rating':[] 
                         })
    
    for id in team.GAME_ID.to_list():
        stats = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id) 
        target_stats[-1]['offensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['offensiveRating'])
        target_stats[-1]['defensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['defensiveRating'])
        target_stats[-1]['net_rating'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['netRating'])
        

In [5]:
np.array(target_stats[0]['defensive_rating']).mean()

119.81379310344828

### Dashboard by Plotly

In [29]:
### Convert the data into dataframe
data = pd.DataFrame(target_stats)
data.offensive_rating = data.offensive_rating.apply(np.array)
data.defensive_rating = data.defensive_rating.apply(np.array)
data.net_rating = data.net_rating.apply(np.array)
data.head()

,team_abbreviation,offensive_rating,defensive_rating,net_rating
0,ATL,"[119.0, 122.0, 128.3, 126.2, 130.9, 104.0, 133...","[125.0, 111.9, 135.4, 120.4, 122.7, 123.8, 125...","[-6.0, 10.1, -7.1, 5.8, 8.2, -19.8, 8.2, -7.1,..."
1,BOS,"[111.3, 124.0, 122.2, 115.2, 129.3, 119.6, 123...","[146.5, 151.6, 117.8, 99.0, 113.3, 111.5, 116....","[-35.1, -27.6, 4.5, 16.2, 16.0, 8.1, 7.2, 8.8,..."
2,CLE,"[101.1, 113.0, 126.5, 131.1, 130.9, 119.6, 123...","[118.5, 130.9, 117.2, 126.2, 122.7, 111.5, 116...","[-17.4, -17.8, 9.4, 4.9, 8.2, 8.1, 7.2, 10.9, ..."
3,NOP,"[110.6, 113.0, 117.7, 100.9, 108.1, 116.2, 119...","[110.4, 130.9, 118.6, 135.2, 112.0, 134.0, 104...","[0.2, -17.8, -0.8, -34.3, -3.9, -17.8, 14.9, 4..."
4,CHI,"[101.1, 116.3, 120.4, 108.3, 129.7, 117.2, 112...","[118.5, 96.0, 104.9, 113.7, 127.5, 126.5, 122....","[-17.4, 20.4, 15.5, -5.4, 2.2, -9.4, -9.8, 2.6..."


In [ ]:
df = data[data.team_abbreviation.isin(['ATL','BOS'])]
df.offensive_rating = df.offensive_rating.apply(np.mean)
df.defensive_rating = df.defensive_rating.apply(np.mean)
df['net_rating'] = df.offensive_rating - df.defensive_rating
display(df.head())

In [51]:
from plotly.subplots import make_subplots

scatter = go.Scatter(
    x=df['offensive_rating'],
    y=df['defensive_rating'],
    mode='markers',
    marker=dict(color='blue', size=8),
    name='Scatter Plot'
)

table = go.Table(
    header=dict(values=list(df.columns),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[df['team_abbreviation'], df['offensive_rating']],
               fill_color='white',
               align='left'),
    domain=dict(x=[0.6, 1],
                y=[0, 1])
)

layout = dict(xaxis1=dict( dict(domain=[0, 0.58], anchor='y1')),
              yaxis1=dict( dict(domain=[0, 1], anchor='x1')),
              margin=dict(l=35, r=35, t=45, b=25),
              title="NBA Teams Statistics"
             )
fig = go.Figure(data = [scatter,table], layout = layout)
iplot(fig, filename = 'basic_table')
